# Question
1. Use RAND to generate synthetic dataset
2. Choose any ANN dataset kaggle or uci machine learning repository

For both, 
1. Implement step-by-step for each with a standard set of weights etc
2. Show tabular representation of hyperparameters as they are tuned at the end. Use CSVLogger code from earlier

Note: <br>
Decide activation functions as well, and define the neural network architecture <br>
Define weights etc, create sub functions for forward and backward propagations <br>
Set a number of iterations and run through the architecture accordingly <br>

#### Imports

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo 

#### ANN Definition

In [20]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def der_sigmoid(x):
    return x * (1 - x)

class my_ANN():
    def __init__(self, learning_rate=0.001, n_hidden_layers=5, weights=None, random_state=42, output_dim=1, neurons_per_layer=[10, 10, 1]):
        self.learning_rate = learning_rate
        self.n_hidden_layers = n_hidden_layers
        self.weights = weights
        self.neurons_per_layer = neurons_per_layer
        self.random_state = random_state
        self.biases = np.random.uniform(size=(output_dim))
        self.loss_history = []
    
    def set_weights(self, x):
        if self.weights is None:
            inp_dim = x.shape[1]
            self.weights = [np.random.uniform(low=-0.01, high=0.01, size=(inp_dim, self.neurons_per_layer[0]))]
            for i in range(1, self.n_hidden_layers):
                self.weights.append(np.random.uniform(low=-0.01, high=0.01, size=(self.neurons_per_layer[i - 1], self.neurons_per_layer[i])))  # Initialize weights for subsequent hidden layers
        return self.weights
    
    def forward_propagation(self, x):
        hidden_layers_output = [x]
        
        for i in range(self.n_hidden_layers):
            dot_prod = np.dot(hidden_layers_output[-1], self.weights[i]) + self.biases
            output = sigmoid(dot_prod)
            hidden_layers_output.append(output)
        
        # final_output = hidden_layers_output[-1]
        final_output = sigmoid(dot_prod)
        return hidden_layers_output, final_output

    def backward_propagation(self, x, y, hidden_layers_output):
        del_weights = [None] * self.n_hidden_layers
        
        # Calculate error
        error = y - hidden_layers_output[-1]

        # Calculate deltas for each layer
        del_output_error = error
        for i in reversed(range(self.n_hidden_layers)):
            del_dot_prod_error = der_sigmoid(hidden_layers_output[i + 1])
            del_weights[i] = np.dot(hidden_layers_output[i].T, del_output_error * del_dot_prod_error)
            del_output_error = np.dot(del_output_error, self.weights[i].T) * del_dot_prod_error

        return del_weights, del_output_error
    
    def cross_entropy_loss(self, y, final_output):
        # Calculate cross-entropy loss
        epsilon = 1e-15  # Small value to prevent division by zero
        final_output = np.clip(final_output, epsilon, 1 - epsilon)
        loss = -np.mean(y * np.log(final_output) + (1 - y) * np.log(1 - final_output))
        return loss

    def train(self, x, y, epochs=30):
        np.random.seed(self.random_state)
        self.weights = self.set_weights(x)

        for _ in range(epochs):
            # Forward propagation
            hidden_layers_output, final_output = self.forward_propagation(x)
            
            # Calculate error
            loss = self.cross_entropy_loss(y, final_output)
            self.loss_history.append(loss)

            # Backward propagation
            del_weights, del_output_error = self.backward_propagation(x, y, hidden_layers_output)
            
            # Updating weights and bias
            for i in range(self.n_hidden_layers):
                self.weights[i] += self.learning_rate * del_weights[i]
                self.biases[i] += self.learning_rate * np.sum(del_output_error * der_sigmoid(hidden_layers_output[i + 1], axis=0))

            self.weights[-1] += self.learning_rate * np.dot(hidden_layers_output[-2].T, del_output_error * der_sigmoid(final_output))
            self.biases[-1] += self.learning_rate * np.sum(del_output_error * der_sigmoid(final_output), axis=0)
            
    def plot_loss(self):
        plt.plot(self.loss_history)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Loss Over epochs')
        plt.show()

#### Synthetic Dataset ANN

In [10]:
n_samples = 13000
n_features = 10
n_classes = 2

x_syn, y_syn = make_classification(n_samples=n_samples, n_features=n_features, n_classes=n_classes, n_informative=6, n_redundant=4, random_state=42)

In [11]:
print(x_syn.shape, type(x_syn.shape))
print(y_syn.shape, type(y_syn.shape), end='\n\n')
print(x_syn[:5])
print(y_syn[:5])

(13000, 10) <class 'tuple'>
(13000,) <class 'tuple'>

[[ 0.32109658 -3.55521745 -1.79901525  5.63529364 -2.88590423 -0.99211446
  -0.82213901  3.08539582  1.79631095  0.69072372]
 [-2.85830434  0.94146699 -0.92897873  0.32050597  1.3548546  -0.58381635
   0.40691861  2.15626725  0.88993014 -0.22135357]
 [-0.19986109  0.11897832  2.01723262  1.93974988 -1.13439803  2.56250543
  -1.75773092 -0.7466832   0.96409535 -3.96289928]
 [-1.2977236   1.00513595  1.31864505  1.02653482 -1.06473218  1.42266209
   1.71876769  0.69682981  0.29560212 -1.4690991 ]
 [ 0.55217032  1.26586226  2.16582174  0.3521321  -0.30993844  0.66955088
   1.84639519  0.34162405 -0.71983337 -0.09763212]]
[1 0 1 1 1]


In [16]:
print(y_syn[:5])

[1 0 1 1 1]


Dataset doesn't need any preprocessing since it is synthetically created specifically for classification

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_syn, y_syn, test_size=0.2, random_state=42)
ann = my_ANN(learning_rate=0.001, n_hidden_layers=3, neurons_per_layer=[10,10,1])

ann.train(x_train, y_train, epochs=20)
_, y_pred = ann.forward_propagation(x_test)
y_pred = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

ValueError: shapes (10400,10400) and (1,10) not aligned: 10400 (dim 1) != 1 (dim 0)

#### Dataset import

In [6]:
# fetch dataset 
dry_bean_dataset = fetch_ucirepo(id=602) 

# data (as pandas dataframes) 
x_uci = dry_bean_dataset.data.features 
y_uci = dry_bean_dataset.data.targets 


In [7]:
print(x_uci.shape)

(13611, 16)


In [8]:
print(x_uci[:1]) 
print(y_uci[:1]) 

    Area  Perimeter  MajorAxisLength  MinorAxisLength  AspectRatio  \
0  28395    610.291       208.178117       173.888747     1.197191   

   Eccentricity  ConvexArea  EquivDiameter    Extent  Solidity  Roundness  \
0      0.549812       28715     190.141097  0.763923  0.988856   0.958027   

   Compactness  ShapeFactor1  ShapeFactor2  ShapeFactor3  ShapeFactor4  
0     0.913358      0.007332      0.003147      0.834222      0.998724  
   Class
0  SEKER
